In [1]:
!python -m pip install --upgrade pip

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple


In [2]:
#! pip install --ignore-installed imageio
! pip install -U albumentations
! pip install timm
! pip install dataclasses

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple


In [3]:
import moxing as mox 
mox.file.copy('obs://lhy3/clothes/clothes_image_all.zip', '/home/ma-user/work/clothes_image_all.zip') # 复制数据到当前路径

INFO:root:Using MoXing-v2.0.0.rc0-19e4d3ab
INFO:root:Using OBS-Python-SDK-3.20.9.1


In [4]:
! unzip -o clothes_image_all.zip

Archive:  clothes_image_all.zip
   creating: clothes_image_all/
   creating: clothes_image_all/training/
  inflating: clothes_image_all/training/1_0.jpg  
  inflating: clothes_image_all/training/20_0.jpg  
  inflating: clothes_image_all/training/10_0.jpg  
  inflating: clothes_image_all/training/20_1.jpg  
  inflating: clothes_image_all/training/9_0.jpg  
  inflating: clothes_image_all/training/0_0.jpg  
  inflating: clothes_image_all/training/1_1.jpg  
  inflating: clothes_image_all/training/15_0.jpg  
  inflating: clothes_image_all/training/0_1.jpg  
  inflating: clothes_image_all/training/0_2.jpg  
  inflating: clothes_image_all/training/0_3.jpg  
  inflating: clothes_image_all/training/18_0.jpg  
  inflating: clothes_image_all/training/0_4.jpg  
  inflating: clothes_image_all/training/6_0.jpg  
  inflating: clothes_image_all/training/11_0.jpg  
  inflating: clothes_image_all/training/10_1.jpg  
  inflating: clothes_image_all/training/4_0.jpg  
  inflating: clothes_image_all/trainin

In [5]:
import os
import glob
import time
import timm
import numpy as np
from PIL import Image
import pandas as pd
import albumentations as A  # 数据增强库
import matplotlib.pyplot as plt

from tqdm import tqdm  # 显示进度条
from albumentations.pytorch import ToTensorV2

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

%matplotlib inline

In [6]:
# 分别读入 training set、test set 的文件路径

dataset_dir = 'clothes_image_all/'    
# dataset_dir = '/path/to/your/dataset/dir'
train_files = glob.glob(os.path.join(dataset_dir, 'training', '*'))  # 返回列表[training目录下所有文件的绝对路径]
test_files = glob.glob(os.path.join(dataset_dir, 'test', '*'))
print(len(train_files), len(test_files))

40871 17517


In [7]:
#training 时做 data augmentation
train_transform = A.Compose([
    #A.HorizontalFlip(p=0.5), # 水平翻转，概率为0.5  
    #A.RandomBrightness(limit=2, p=0.5),#随机提升亮度
    #A.ElasticTransform(alpha=155, sigma=210, alpha_affine=157, p=0.5),
    A.Resize(224, 224, always_apply=True),  # 将不同尺寸的图像缩放到同一尺寸
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),  # 使用imagenet的均值和方差进行normalization
    ToTensorV2(), # 将numpy转换为Tensor， 并将图像（H,W,3）转换为（3,H,W）
])  # 其他方式的数据增强请参考官方文档

#testing 时候不需做 data augmentation
test_transform = A.Compose([
    A.Resize(224, 224, always_apply=True),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

class ImgDataset(Dataset):
    def __init__(self, files_list, transform=None, mode='train'):
        self.files_list = files_list
        self.transform = transform
        self.mode = mode
    
    def __getitem__(self, index):
        file = self.files_list[index]
        image = np.array(Image.open(file))
        
        if self.transform is not None:
            aug = self.transform(image=image)
            image = aug['image']
        

        if self.mode in ['train', 'test']: 
            label = int(os.path.basename(file).split('_')[0])
            return image, label

        else:
            return image
        
    def __len__(self):
        return len(self.files_list)

In [8]:
batch_size = 64
train_set = ImgDataset(train_files, train_transform, mode='train')
test_set = ImgDataset(test_files, test_transform, mode='test')
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=4)

In [16]:
mox.file.copy('obs://lhy3/homework/resnet34-43635321.pth', '/home/ma-user/.cache/torch/checkpoints/resnet34-43635321.pth') # 复制数据到当前路径

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = timm.create_model("resnet34", pretrained=True, num_classes=23, drop_rate=0.).to(device)

In [18]:
model = model.to(device)  # 如果有GPU则在GPU上运行, 否则为CPU
loss = nn.CrossEntropyLoss() # 因为是 classification task，所以 loss 使用 CrossEntropyLoss
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) # optimizer 使用 Adam
num_epoch = 10

train_acc_list = []
train_loss_list = []
test_acc_list = []
test_loss_list = []
true_label = []
pred_label =[]

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0
    test_acc = 0.0
    test_loss = 0.0

    model.train() # 确保 model 是在 train model (开启 Dropout 等...)
    for image, label in tqdm(train_loader):
        optimizer.zero_grad() # 用 optimizer 将 model 参数的 gradient 置零
        
        image = image.to(device)  # 如果有GPU则在GPU上运行
        label = label.to(device)
        
        train_pred = model(image) # 利用 model 得到预测的概率分布 这边实际上就是去呼叫 model 的 forward 函数
        batch_loss = loss(train_pred, label) # 计算 loss （注意 prediction 跟 label 必须同时在 CPU 或是 GPU 上）
        batch_loss.backward() # 利用 back propagation 算出每個参数的 gradient
        optimizer.step() # 以 optimizer 用 gradient 更新参数值
        
        train_acc += torch.sum(torch.argmax(train_pred, dim=1) == label).item()
        train_loss += batch_loss.item()
       
    
    model.eval()
    with torch.no_grad():
        for image, label in tqdm(test_loader):
            image = image.to(device)
            label = label.to(device)
            if epoch==9:
                true_label.append(list(label.cpu().numpy()))
            
            test_pred = model(image)
            batch_loss = loss(test_pred, label)
            
            if epoch==9:
                pred_label.append(list(torch.argmax(test_pred,dim=1).cpu().numpy()))

            test_acc += torch.sum(torch.argmax(test_pred, dim=1) == label).item()
            test_loss += batch_loss.item()

    
    train_acc_list.append(train_acc / train_set.__len__())
    train_loss_list.append(train_loss / train_set.__len__())
    test_acc_list.append(test_acc / test_set.__len__())
    test_loss_list.append(test_loss / test_set.__len__())
    
        #將結果 print 出來
    print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Test Acc: %3.6f loss: %3.6f' % \
        (epoch + 1, num_epoch, time.time()-epoch_start_time, \
         train_acc/train_set.__len__(), train_loss/train_set.__len__(), test_acc/test_set.__len__(), test_loss/test_set.__len__()))

  0%|          | 0/638 [00:00<?, ?it/s]

[001/010] 148.35 sec(s) Train Acc: 0.695603 Loss: 0.015112 | Test Acc: 0.779072 loss: 0.010526


  0%|          | 0/638 [00:00<?, ?it/s]

[002/010] 147.66 sec(s) Train Acc: 0.861197 Loss: 0.006787 | Test Acc: 0.811326 loss: 0.009298


  0%|          | 0/638 [00:00<?, ?it/s]

...

[008/010] 148.72 sec(s) Train Acc: 0.978909 Loss: 0.000988 | Test Acc: 0.835303 loss: 0.011018


  0%|          | 0/638 [00:00<?, ?it/s]

[009/010] 149.96 sec(s) Train Acc: 0.982457 Loss: 0.000806 | Test Acc: 0.831763 loss: 0.011385


100%|██████████| 274/274 [00:24<00:00, 11.24it/s]

[010/010] 148.85 sec(s) Train Acc: 0.981332 Loss: 0.000851 | Test Acc: 0.822344 loss: 0.011980


In [ ]:
state_dict = model_best.state_dict()
torch.save(state_dict, 'model_best.pth')

In [ ]:
mox.file.copy( '/home/ma-user/work/model_best.pth','obs://lhy3/homework/model_best.pth') # 把模型保存至obs中，在home page即可下载